# Файл отладки

In [16]:
import os
from pathlib import Path

import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torchsummary import summary
import torchvision.transforms.v2 as transforms_v2
from torchvision.transforms.v2 import InterpolationMode

# Import Datasets.
from src.datasets.TinyImageNetDataset import TinyImageNetDataset
from src.datasets.MoonSegmentBinaryDataset import MoonSegmentationDataset

# Import models.
from src.models.customResNet import customResNet
from src.models.customUNet import customUNet
from src.models.customResNetUNet import customResNetUNet

from src.main import set_seed
from src.train import MetricsHistory

In [17]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version used by PyTorch: {torch.version.cuda}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

CUDA is not available. PyTorch is using CPU.


Пути к файлам конфигурации.

In [18]:
hyperparameters_dir = Path("./src/hyperparameters/")

config_path = hyperparameters_dir / "config.json"
assert config_path.exists(), f"Config not found: {config_path}"
with open(config_path, "r") as f:
    general_config = json.load(f)

checkpoints_dir = Path(general_config["checkpoints_dir"])
logs_dir = Path(general_config["logs_dir"])

Seed и device.

In [19]:
set_seed(general_config["seed"])
device = torch.device(general_config["device"].lower() if torch.cuda.is_available() else 'cpu')

## Отладка датадоудеров

In [20]:
from enum import IntEnum

# user-defined index.
class dataid(IntEnum):
    tiny_imagenet_200 = 0
    moon_segmentation_binary = 1

dataset_names = [e.name for e in dataid]

In [21]:
mdl_input_size = [3, 128, 128]

mean_norm = [0.485, 0.456, 0.406]
std_norm = [0.229, 0.224, 0.225]

In [22]:
for name in dataset_names:
    if name == "tiny_imagenet_200":
        name = "tiny-imagenet-200"
    elif name == "moon_segmentation_binary":
        name = "moon-segmentation-binary"
    else:
        raise NotImplementedError(f"Dataset not supported: {name}")
    
    data_path = Path(general_config.get("data_dir")) / name
    
    dataset_config_path = hyperparameters_dir / (name + "-config.json")
    assert dataset_config_path.exists(), f"Config not found: {dataset_config_path}"
    with open(dataset_config_path, "r") as f:
        dataset_config = json.load(f)
    
    if name == "tiny-imagenet-200":
        val_augmentations = transforms_v2.Compose([
            transforms_v2.ToImage(),
            transforms_v2.ToDtype(torch.float32, scale=True),
            transforms_v2.Resize(
                tuple(mdl_input_size[-2:]),
                interpolation=InterpolationMode.NEAREST,
                antialias=True
            )
        ])

        postprocessing = transforms_v2.Compose([
            transforms_v2.Normalize(mean=mean_norm, std=std_norm)
        ])

        dataset = TinyImageNetDataset(
            data_path = data_path,
            split = "val",
            augmentations = val_augmentations,
            postprocessing = postprocessing,
            selected_classes =dataset_config['selected_classes']
        )

    elif name == "moon-segmentation-binary":
        data_path = data_path / 'images'
        
        img_prefix = dataset_config['img_prefix']
        mask_prefix = dataset_config['mask_prefix']
        img_folder = img_prefix + '/'
        all_images = [img_no_ext.replace(img_prefix, '') for img_no_ext in
            [img.replace('.png', '') for img in sorted(os.listdir(data_path / img_folder)) if img.endswith('.png')]
        ]
        
        train_images, val_images = train_test_split(
            all_images,
            test_size=0.2,
            random_state = general_config.get('seed')
        )
    
        val_augmentations = transforms_v2.Compose([
            transforms_v2.Resize(
                size=tuple(mdl_input_size[-2:]),
                interpolation=InterpolationMode.NEAREST,
                antialias=True
            ),
        ])

        postprocessing = transforms_v2.Compose([
            transforms_v2.Normalize(mean=mean_norm, std=std_norm),
        ])

        dataset = MoonSegmentationDataset(
                data_path = data_path,
                samples = val_images,
                img_prefix = img_prefix,
                mask_prefix = mask_prefix,
                geometric_augmentations = val_augmentations,
                photometric_augmentations = None,
                postprocessing = postprocessing
            )
        
    else:
        raise NotImplementedError(f"Dataset not supported: {name}")

In [8]:
dataset.__getitem__(0)[0].dtype

torch.float32

## Отладка моделей
### ResNet

In [9]:
model_name = "customResNet"

model_config_path = hyperparameters_dir / f"{model_name}-config.json"
assert model_config_path.exists(), f"Config not found: {model_config_path}"
with open(model_config_path, "r") as f:
    model_config = json.load(f)

dataset_config_path = hyperparameters_dir / f"{model_config.get('dataset_name')}-config.json"
assert dataset_config_path.exists(), f"Config not found: {dataset_config_path}"
with open(dataset_config_path, "r") as f:
    dataset_config = json.load(f)
    
data_path = Path(general_config.get("data_dir")) / model_config.get("dataset_name")

checkpoints_file = checkpoints_dir / f"best_{model_name}.pth"
assert checkpoints_file.exists(), f"Checkpoints not found: {checkpoints_file}"

selected_classes = dataset_config.get("selected_classes")
n_classes = len(selected_classes)

mdl_input_size = model_config.get('input_size')

In [10]:
if checkpoints_file is None:
        pretrained_flag = False
else:
        pretrained_flag = True

model, _, _ = customResNet(
        layers_config = model_config.get("layers_num")*[model_config.get("block_size")],
        in_channels = mdl_input_size[0],
        layer0_channels = model_config.get("output_channels") // 2**(model_config.get("layers_num") - 1),
        num_classes = n_classes,
        pretrained = pretrained_flag,
        checkpoints_file = checkpoints_file,
        device = device
        )
model.eval()

test_input = torch.randn(1, *mdl_input_size).to(device)
print(model_config.get("layers_num")*[model_config.get("block_size")])
model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model size: {model_size}")

summary(model, tuple(mdl_input_size))

Restoring checkpoint:  checkpoints\best_customResNet.pth
[2, 2, 2, 2]
Model size: 889732
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 18, 64, 64]           2,646
       BatchNorm2d-2           [-1, 18, 64, 64]              36
              ReLU-3           [-1, 18, 64, 64]               0
         MaxPool2d-4           [-1, 18, 64, 64]               0
            Conv2d-5           [-1, 18, 32, 32]             324
       BatchNorm2d-6           [-1, 18, 32, 32]              36
            Conv2d-7           [-1, 18, 32, 32]           2,916
       BatchNorm2d-8           [-1, 18, 32, 32]              36
              ReLU-9           [-1, 18, 32, 32]               0
           Conv2d-10           [-1, 18, 32, 32]           2,916
      BatchNorm2d-11           [-1, 18, 32, 32]              36
             ReLU-12           [-1, 18, 32, 32]               0
       BasicBl

### U-Net

In [11]:
model_name = "customUNet"

model_config_path = hyperparameters_dir / f"{model_name}-config.json"
assert model_config_path.exists(), f"Config not found: {model_config_path}"
with open(model_config_path, "r") as f:
    model_config = json.load(f)

dataset_config_path = hyperparameters_dir / f"{model_config.get('dataset_name')}-config.json"
assert dataset_config_path.exists(), f"Config not found: {dataset_config_path}"
with open(dataset_config_path, "r") as f:
    dataset_config = json.load(f)
    
data_path = Path(general_config.get("data_dir")) / model_config.get("dataset_name")

checkpoints_file = checkpoints_dir / f"best_{model_name}.pth"
assert checkpoints_file.exists(), f"Checkpoints not found: {checkpoints_file}"

mdl_input_size = model_config.get('input_size')

In [12]:
model = customUNet(
    in_channels = mdl_input_size[0],
    out_channels = 1,
    features = model_config.get('feature_list')
    )
model = model.to(device)
model.eval()

test_input = torch.randn(1, *mdl_input_size).to(device)
test_output = model(test_input)

model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model size: {model_size}")
print(f"   Вход:  {test_input.shape}")
print(f"   Выход: {test_output.shape}")
summary(model, tuple(mdl_input_size))

Encoder features by level: [18, 36, 72, 144]
Model size: 2459719
   Вход:  torch.Size([1, 3, 128, 128])
   Выход: torch.Size([1, 1, 128, 128])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 18, 128, 128]             504
       BatchNorm2d-2         [-1, 18, 128, 128]              36
              ReLU-3         [-1, 18, 128, 128]               0
            Conv2d-4         [-1, 18, 128, 128]           2,934
       BatchNorm2d-5         [-1, 18, 128, 128]              36
              ReLU-6         [-1, 18, 128, 128]               0
        DoubleConv-7         [-1, 18, 128, 128]               0
         MaxPool2d-8           [-1, 18, 64, 64]               0
            Conv2d-9           [-1, 36, 64, 64]           5,868
      BatchNorm2d-10           [-1, 36, 64, 64]              72
             ReLU-11           [-1, 36, 64, 64]               0
           Conv2d-12    

### U-Net + backbone

In [13]:
model = customResNetUNet(
    in_channels = mdl_input_size[0],
    out_channels = 1,
    features = model_config.get('feature_list'),
    backbone_layers_config = 4*[2],
    backbone_layer0_channels = model_config.get('feature_list')[0],
    backbone_pretrained = False,
    backbone_checkpoints_file = None,
    device = device
    )
model = model.to(device)
model.eval()

test_input = torch.randn(1, 3, 128, 128).to(device)
test_output = model(test_input)

model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model size: {model_size}")
print(f"   Вход:  {test_input.shape}")
print(f"   Выход: {test_output.shape}")
summary(model, tuple(mdl_input_size))

Encoder features by level: [18, 36, 72, 144]
Model size: 2963395
   Вход:  torch.Size([1, 3, 128, 128])
   Выход: torch.Size([1, 1, 128, 128])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 18, 64, 64]           2,646
       BatchNorm2d-2           [-1, 18, 64, 64]              36
              ReLU-3           [-1, 18, 64, 64]               0
         MaxPool2d-4           [-1, 18, 64, 64]               0
            Conv2d-5           [-1, 36, 32, 32]             648
       BatchNorm2d-6           [-1, 36, 32, 32]              72
            Conv2d-7           [-1, 36, 32, 32]           5,832
       BatchNorm2d-8           [-1, 36, 32, 32]              72
              ReLU-9           [-1, 36, 32, 32]               0
           Conv2d-10           [-1, 36, 32, 32]          11,664
      BatchNorm2d-11           [-1, 36, 32, 32]              72
             ReLU-12    